In [ ]:
import os
import sys, getopt
import os.path

In [6]:
def Annotation_prepar_3UTR_extraction(gene_bed_file, gene_symbol_map_kfXref_file, output_utr_file):
    
    output_write = open(output_utr_file,'w')
    
    refseq_trapt_gene_symbol_dict = {}
    num_line = 0
    for line in open(gene_symbol_map_kfXref_file, 'r'):
        if num_line > 0:
            fields = line.strip('\n').strip('\r').split('\t')
            gene_symbol = fields[1]
            refseq_transcript_id = fields[0]
            refseq_trapt_gene_symbol_dict[refseq_transcript_id] = gene_symbol
        else:
            num_line += 1
    
    scanned_3UTR_list = []
    num_saved = 0
    for line in open(gene_bed_file,'r'):
        fields = line.strip('\n').split('\t')
        refseq_id = fields[3]
        if '_' not in fields[0]:
            
            if refseq_id not in refseq_trapt_gene_symbol_dict:
                gene_symbol = "NA"
            else:
                gene_symbol = refseq_trapt_gene_symbol_dict[refseq_id] 
            
            UTR_id = [refseq_id, gene_symbol,fields[0], fields[5]]
            UTR_id_new = '|'.join(UTR_id)
            curr_strand = fields[5]
            if curr_strand == "+":
                UTR_end = fields[2]
                gene_start = int(fields[1])
                UTR_start = str(gene_start + int(fields[-1].strip(',').split(',')[-1])+1)#1base
            elif curr_strand == "-":
                gene_start = int(fields[1])
                UTR_start = str(gene_start + 1)#1base
                UTR_end   = str(gene_start + int(fields[10].split(',')[0]))#1base, included
            
            this_UTR = fields[0]+UTR_start+UTR_end+curr_strand
            if this_UTR not in scanned_3UTR_list:
                write_line = [fields[0], UTR_start, UTR_end,UTR_id_new, '0', curr_strand]
                output_write.writelines('\t'.join(write_line) + '\n')
                scanned_3UTR_list.append(this_UTR)
                num_saved += 1
    
    
    output_write.close()   
    print("Total extracted 3' UTR: " + str(num_saved))

In [ ]:
gene_bed_file='DATA/hg38_wholeGene_annotation.bed'
gene_symbol_annotation_file='DATA/hg38_refseq_IDmapping.txt'
output_extract_file = 'temp_anno_extracted.bed'




output_final_extract_file='DATA/hg38_refseq_IDmapping.txt'

Annotation_prepar_3UTR_extraction(
    gene_bed_file, 
    gene_symbol_annotation_file,
    output_extract_file)

In [ ]:
num_line = 0
gene_symbol_annotation_file='DATA/hg38_refseq_IDmapping.txt'
for line in open(gene_symbol_annotation_file, 'r'):
    print(line)
    if num_line > 0:
        break
    num_line += 1

#name	name2

NM_001276352.2	C1orf141



In [13]:
fields = line.strip('\n').strip('\r').split('\t')
fields

['NM_001276352.2', 'C1orf141']

In [16]:
refseq_trapt_gene_symbol_dict = {}
type(refseq_trapt_gene_symbol_dict)

dict

In [18]:
gene_symbol = fields[1]
refseq_transcript_id = fields[0]
refseq_trapt_gene_symbol_dict[refseq_transcript_id] = gene_symbol
refseq_trapt_gene_symbol_dict

{'NM_001276352.2': 'C1orf141'}

In [21]:
scanned_3UTR_list = []
num_saved = 0


gene_bed_file='DATA/hg38_wholeGene_annotation.bed'
for line in open(gene_bed_file,'r'):
    print(line)
    break

chr1	67092164	67134970	NM_001276352.2	0	-	67093579	67127240	0	9	1440,70,145,68,113,158,92,86,41,	0,4087,11073,19412,23187,33587,35001,38977,42765,



In [23]:
fields = line.strip('\n').split('\t')
fields

['chr1',
 '67092164',
 '67134970',
 'NM_001276352.2',
 '0',
 '-',
 '67093579',
 '67127240',
 '0',
 '9',
 '1440,70,145,68,113,158,92,86,41,',
 '0,4087,11073,19412,23187,33587,35001,38977,42765,']

In [24]:
refseq_id = fields[3]
refseq_id

'NM_001276352.2'

In [25]:
if refseq_id not in refseq_trapt_gene_symbol_dict:
    gene_symbol = "NA"
else:
    gene_symbol = refseq_trapt_gene_symbol_dict[refseq_id] 
            

In [26]:
UTR_id = [refseq_id, gene_symbol,fields[0], fields[5]]
UTR_id

['NM_001276352.2', 'C1orf141', 'chr1', '-']

In [27]:
UTR_id_new = '|'.join(UTR_id)
UTR_id_new

'NM_001276352.2|C1orf141|chr1|-'

In [28]:
curr_strand = fields[5]
curr_strand

'-'

In [29]:
if curr_strand == "+":
    UTR_end = fields[2]
    gene_start = int(fields[1])
    UTR_start = str(gene_start + int(fields[-1].strip(',').split(',')[-1])+1)#1base
elif curr_strand == "-":
    gene_start = int(fields[1])
    UTR_start = str(gene_start + 1)#1base
    UTR_end   = str(gene_start + int(fields[10].split(',')[0]))#1base, included


In [30]:
UTR_end

'67093604'

In [32]:
fields[0]

'chr1'

In [33]:
UTR_start

'67092165'

In [31]:
this_UTR = fields[0]+UTR_start+UTR_end+curr_strand
this_UTR

'chr16709216567093604-'

In [35]:
temp_file = "overlap_opposite_strand_subtract.bed"
cmd = 'subtractBed -a %s -b %s -S > %s' % (output_extract_file, output_final_extract_file, temp_file)
os.system(cmd)

Error: unable to open file or unable to determine types for file DATA/hg38_refseq_IDmapping.txt

- Please ensure that your file is TAB delimited (e.g., cat -t FILE).
- Also ensure that your file has integer chromosome coordinates in the 
  expected columns (e.g., cols 2 and 3 for BED).


256

In [39]:
def Subtract_different_strand_overlap(input_gene_bed_file,output_utr_file):
    def UTRs_subtract_refine(UTRs_all):
        strand_info = UTRs_all[0].strip('\n').split('\t')[-1]
        if strand_info == '+':
            all_pos = []
            for curr_line in UTRs_all:
                left_pos = curr_line.strip('\n').split('\t')[1]
                all_pos.append(int(left_pos))
            selected_UTR_index = all_pos.index(min(all_pos))
            selected_UTR = UTRs_all[selected_UTR_index]
        else:
            all_pos = []
            for curr_line in UTRs_all:
                left_pos = curr_line.strip('\n').split('\t')[2]
                all_pos.append(int(left_pos))
            selected_UTR_index = all_pos.index(max(all_pos))
            selected_UTR = UTRs_all[selected_UTR_index]
        return selected_UTR


    temp_file = "overlap_opposite_strand_subtract.bed"
    cmd = 'subtractBed -a %s -b %s -S > %s' % (input_gene_bed_file, input_gene_bed_file, temp_file)

    print('cmd',cmd)


    os.system(cmd)
    
    read_subtract_result_dict = {}
    for line in open(temp_file,'r'):
        transcript_id = line.split('\t')[3].split('|')[0]
        if transcript_id not in read_subtract_result_dict:
            read_subtract_result_dict[transcript_id] = []
        read_subtract_result_dict[transcript_id].append(line)
    
    output_utr_write = open(output_utr_file,'w')
    for curr_trans_id in read_subtract_result_dict:
        curr_3UTRs = read_subtract_result_dict[curr_trans_id]
        num_3UTRs = len(curr_3UTRs)
        if num_3UTRs == 1:
            output_utr_write.writelines(curr_3UTRs[0])
        else:
            selected_UTR = UTRs_subtract_refine(curr_3UTRs)
            output_utr_write.writelines(selected_UTR)
    output_utr_write.close()
    



In [40]:
Subtract_different_strand_overlap(output_extract_file,output_final_extract_file)
    

cmd subtractBed -a temp_anno_extracted.bed -b temp_anno_extracted.bed -S > overlap_opposite_strand_subtract.bed
